<a href="https://colab.research.google.com/github/shubhamg0510/DeepChem/blob/master/SortNet1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from __future__ import print_function
import sys
import torch
#from PIL import Image
import matplotlib.pyplot as pyplot
import numpy as np
from torch.autograd import Variable as V
from torchvision import transforms, datasets, models
from torch.autograd import Variable
from torch.autograd import Function as F
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import time
import pandas as pd
#from Datasets import Amazon_specific_item_pairs
from sklearn.metrics import roc_auc_score

In [0]:
from torch.utils.data import Dataset
import torch
import os
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import cv2
from random import sample
from scipy import stats
import torch.nn.functional as F
from random import seed
#seed(4523)
import os

class Amazon_specific_item_pairs(Dataset):
    def __init__(self, root_dir, sample_type, img_size, batch_size):
        self.root_dir = root_dir
        self.item1_img_files = []
        self.item2_img_files = []
        self.img_size = img_size
        self.attractive_img_files=[]
        self.sample_type = sample_type

        if(sample_type=='train'):
          data = pd.read_csv("/content/drive/My Drive/Amazon_Same_Category_Pairs.csv",nrows=20000)
        else:
          data = pd.read_csv("/content/drive/My Drive/Amazon_Same_Category_Pairs.csv", nrows=9000)


        # data = data.reset_index(drop=True)
        # nb_rows = int(np.floor(len(data)/batch_size)*batch_size)
        # data = data.sample(n=nb_rows)
        # data = data.reset_index(drop=True)
        # self.item1_img_files = list(data["item1_img_url"])
        # self.item2_img_files = list(data["item2_img_url"])

        # self.attractive_img_files = list(data['attractive_item_img_url'])
        # # self.sample_type = list(data['sample'])
        self.a = pd.read_csv("/content/drive/My Drive/Amazon_Same_Category_Pairs.csv")
        self.b=self.a.item1_img_url
        self.c=self.a.item2_img_url
        self.d=self.a.attractive_item_img_url
        self.b=self.b[:4000]
        self.c=self.c[:4000]
        self.d=self.d[:4000]
        self.b1=[q.split('/')[7] for q in self.b]
        self.c1=[q.split('/')[7] for q in self.c]
        self.d1=[q.split('/')[7] for q in self.d]
        
    def __len__(self):
        return len(self.d)

    def getvec(self,add,prefix):
        img = Image.open(prefix+add)
        img_t = transform(img)
        batch_t = torch.unsqueeze(img_t, 0)
        return batch_t

    def __getitem__(self,idx):
        # base=os.path.basename(self.item1_img_files[idx])
        # item1_img = cv2.imread("/content/drive/My Drive/images/"+base)
        # base=os.path.basename(self.item2_img_files[idx])
        # item2_img = cv2.imread("/content/drive/My Drive/images/"+base)
        # base=os.path.basename(self.attractive_img_files[idx])
        # attractive_img = cv2.imread("/content/drive/My Drive/images/"+base)
        
        # item1_img = cv2.resize(item1_img, (50,50))
        # item2_img = cv2.resize(item2_img, (50,50))
        # attractive_img = cv2.resize(attractive_img, (50,50))
        # item1_img = self.resizeAndPad(,item1_img,self.img_size)
        # item2_img = self.resizeAndPad(,item2_img,self.img_size)
        # attractive_img = self.resizeAndPad(,attractive_img,self.img_size)
        item1_img=self.getvec(self.b1[idx],"/content/drive/My Drive/images/")
        item2_img=self.getvec(self.c1[idx],"/content/drive/My Drive/images/")
        attractive_img=self.getvec(self.d1[idx],"/content/drive/My Drive/images/")


        # label = self.attractive_img_files[idx]
        # sample_type = self.sample_type
        return item1_img, item2_img, attractive_img


In [0]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias.data)

In [0]:
class SortNet(nn.Module):
    def __init__(self, net_cnn):
        super(SortNet, self).__init__()
        self.img_encoder = nn.Sequential(*list(net_cnn.children())[:-1])
        self.layer_1A = nn.Sequential(nn.Linear(2048,512,bias = True))
        self.layer_1B = nn.Sequential(nn.Linear(2048,512,bias = True))
        self.sig    = nn.Sigmoid()
        self.layer_2A = nn.Sequential(nn.Linear(512,1,bias = True))
        self.layer_2B = nn.Sequential(nn.Linear(512,1,bias = True))
        
    def forward(self,item1,item2):
        print("fstart")
        item1 = torch.squeeze(item1)
        item1 = self.img_encoder(item1)
        item2 = torch.squeeze(item2)
        item2 = self.img_encoder(item2)

        print("forward1")
        #layer one
        x_layer_1A = self.layer_1A(item1)
        x_layer_1B = self.layer_1B(item1)
        y_layer_1A = self.layer_1A(item2)
        y_layer_1B = self.layer_1B(item2)
        #left node of first layer
        h_right_xy = torch.cat([x_layer_1A,y_layer_1B],dim=0)
        h_right_xy = self.sig(h_right_xy)
        #right node of first layer
        h_left_xy = torch.cat([x_layer_1B + y_layer_1A],dim=0)
        h_left_xy = self.sig(h_left_xy)
        print("forward2")
        #layer two
        p = h_right_xy
        q = h_left_xy
        
        p_layer_2A = self.layer_2A(p)
        p_layer_2B = self.layer_2B(p)
        q_layer_2A = self.layer_2A(q)
        q_layer_2B = self.layer_2B(q)
            
        n_right_xy = torch.cat([p_layer_2A + q_layer_2B],dim=0)
        n_right_xy = self.sig(n_right_xy)
            
        n_left_xy = torch.cat([p_layer_2B + q_layer_2A],dim=0)
        n_left_xy = self.sig(n_left_xy)
        print("forward3")   
        return n_right_xy,n_left_xy

In [6]:
net_cnn = models.resnet18(pretrained=True)
net = SortNet(net_cnn)
net.layer_1A.apply(weights_init)
net.layer_1B.apply(weights_init)
net.layer_2A.apply(weights_init)
net.layer_2B.apply(weights_init)

Sequential(
  (0): Linear(in_features=512, out_features=1, bias=True)
)

In [0]:
num_classes = 1
iterations = 100
lr = 0.00005
BatchSize = 256
IMG_SIZE=224
path = "/home/debopriyo/ng_sural/data/BOXREC/"
#transform = transforms.Compose([transforms.ToTensor()])
# # transform = transforms.Compose([transforms.ToPILImage(),
#                                 transforms.RandomHorizontalFlip(),
#                                 transforms.ToTensor(),
#                                 transforms.Normalize([0.406, 0.456, 0.485], 
#                                                      [ 0.225, 0.224, 0.229])])
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])


In [8]:
trainset = Amazon_specific_item_pairs(root_dir=path,sample_type='train', img_size = IMG_SIZE, batch_size = BatchSize)
trainset_len = trainset.__len__()
print('Trainset Length :', trainset_len)
trainsetloader = torch.utils.data.DataLoader(trainset,batch_size=BatchSize,shuffle=True)
                           
valset = Amazon_specific_item_pairs(root_dir=path,sample_type='val', img_size = IMG_SIZE, batch_size = BatchSize)
valset_len = valset.__len__()
print('valset Length :', valset_len)
valsetloader = torch.utils.data.DataLoader(valset,batch_size=BatchSize,shuffle=False)

Trainset Length : 4000
valset Length : 4000


In [9]:
optimizer = optim.Adam(net.parameters(), lr = lr, weight_decay=0.00005)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1)
#%%
use_gpu = torch.cuda.is_available()
#use_gpu = False
if use_gpu :
    print('GPU is available...')
    sys.stdout.flush()
    net = net.cuda()
    resnet18_model = resnet18_model.cuda()
else:
    print('GPU not available...')
    sys.stdout.flush()

#net_chkpt = torch.load('/home/debopriyo/ng_sural/data/DMLFCAD_'+pair_type+'/models/DMLFCAD_tw_bw_Model_Epoch_19_TrainAcc_84.412202_ValAcc_82.34375_0.748691.pth',map_location='cpu')
#
#net.load_state_dict(net_chkpt['state_dict'])

#criterion = nn.BCELoss()
criterion = nn.NLLLoss()
# with open('/home/debopriyo/ng_sural/data/DMLFCAD_'+pair_type+'/models/DMLFCAD_'+pair_type+'_Model_Training_Validation_stats_lr_'+str(lr)+'.csv','w') as f:
#     f.write('Epoch,LearningRate,AvgTrainLoss,AvgTrainAcc,Train_AUC,AvgValLoss,AvgValAcc,Val_AUC\n')

GPU not available...


In [0]:
start = time.time()
valAcc = []
trainLoss = []
valLoss = []
for epoch in range(iterations):
    
    epochStart = time.time()
    tr_runningLoss = 0.0
    val_runningLoss = 0.0
    tr_running_correct = 0
    predicted_y = []
    ground_truth_y = []
    print("Epoch:",epoch+1,"started...")
    sys.stdout.flush()
    for i, data in enumerate(trainsetloader, 0):
        print("starting tr")
        net.train(True)
        # resnet18_model.train(False)
        item1_inputs, item2_inputs, labels = data


        item1_inputs = item1_inputs.float()
        item2_inputs = item2_inputs.float()

        # wrap them in variable
        if use_gpu:
            item1_inputs, item2_inputs = Variable(item1_inputs.cuda()), Variable(item2_inputs.cuda())
            labels = V(labels.cuda())
  
        else:
            item1_inputs, item2_inputs = Variable(item1_inputs), Variable(item2_inputs)
            labels = V(labels)

        features_blobs = []
        optimizer.zero_grad() # zeroes the gradient buffers of all parameters
        
        outputs = net(item1_inputs, item2_inputs) # forward
        print("pos 1")
#        print(outputs.shape)
        loss = criterion(torch.squeeze(F.sigmoid(outputs)), labels.float()) # calculate loss


        loss.backward() # backpropagate the loss

        optimizer.step()
        
        tr_runningLoss += loss.item()
        
        labels = labels.long()
        print("pos 2")
        # output1=outputs[0]
        # output2=outputs[1]
        # if(output1>output2):
        #   predicted_img=item1_inputs
        #   predicted_num=0
        # else:
        #   predicted_img=item2_inputs
        #   predicted_num=1
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        tr_running_correct += ((predicted==labels).sum()).item()
        
        predicted = predicted.cpu()
        labels = labels.cpu()
        
#        predicted = predicted.long()
#        labels = labels.long()
        predicted_y += list(predicted.numpy())
        ground_truth_y += list(labels.numpy())
        
        
        print("Epoch :{:.0f} ; Batch : {:.0f} ; Training Loss : {:.6f} ; Training Accuracy : {:.6f}".format(epoch+1, i+1, tr_runningLoss/(i+1),(tr_running_correct/((i+1)*BatchSize))*100))
        sys.stdout.flush()
    tr_auc = roc_auc_score(np.array(ground_truth_y),np.array(predicted_y))
    avgTrainLoss = tr_runningLoss/(trainset_len/BatchSize)
    avgTrainAcc = (tr_running_correct/(trainset.__len__()))*100
    print("Epoch :{:.0f} ; Average Train Accuracy : {:.6f} ; Training AUC : {:.6f} ;".format(epoch+1,avgTrainAcc,tr_auc))
    sys.stdout.flush()
    trainLoss.append(avgTrainLoss)

    predicted_y = []
    ground_truth_y = []
    del predicted
    del labels
    print('Starting Validation...')
    sys.stdout.flush()
    with torch.no_grad():
        net.train(False) # For validation
#        resnet18_model.train(False)
        val_running_correct = 0
       
        for i, data in enumerate(valsetloader, 0):
            item1_inputs, item2_inputs, labels = data

            item1_inputs = item1_inputs.float()
            item2_inputs = item2_inputs.float()

            # Wrap them in Vriable
            if use_gpu:
                item1_inputs, item2_inputs = Variable(item1_inputs.cuda()), Variable(item2_inputs.cuda())
                labels = V(labels.cuda())

            else:
                item1_inputs, item2_inputs = Variable(item1_inputs), Variable(item2_inputs)
                labels = V(labels)

            
            features_blobs = []
            
            outputs = net(item1_inputs, item2_inputs)
            val_loss = criterion(torch.squeeze(F.sigmoid(outputs)), labels.float()) # calculate loss
            
            
            #predicted = torch.squeeze(F.sigmoid(outputs.data)>0.5).long()
            _, predicted = torch.max(outputs.data, 1)
            labels = labels.long()
                
            
            val_running_correct += ((predicted == labels).sum()).item()
            val_runningLoss += val_loss.item()
            
            predicted = predicted.cpu()
            labels = labels.cpu()
            
#            predicted = predicted.long()
            
            
            predicted_y += list(predicted.numpy())
            ground_truth_y += list(labels.numpy())
            
        avgValAcc = (val_running_correct/(valset.__len__()))*100
        avgValLoss = val_runningLoss/(valset_len/BatchSize)
        
    scheduler.step(avgValLoss)
    val_auc = roc_auc_score(np.array(ground_truth_y),np.array(predicted_y))
    valAcc.append(avgValAcc)
    print('Val Accuracy :',avgValAcc)
    epochEnd = time.time()-epochStart
    print('Iteration: {:.0f}/{:.0f} ; Training Loss: {:.6f} ; Training Accuracy : {:.6f} ; Training AUC : {:.6f} ; Validation Loss: {:.6f} ; Validation Accuracy: {:.6f} ; Validation AUC : {:6f} ; Time Consumed: {:.0f}m {:.0f}s'.
          format(epoch + 1, iterations, avgTrainLoss, avgTrainAcc, tr_auc, avgValLoss, avgValAcc, val_auc, epochEnd//60, epochEnd%60))
    
    sys.stdout.flush()
    model_path = "/home/debopriyo/ng_sural/data/DMLFCAD_"+pair_type+"/models/DMLFCAD_"+pair_type+"_Model_Epoch_"+\
                    str(epoch+1)+"_TrainAcc_"+str(round(avgTrainAcc,6))+"_ValAcc_"+\
                    str(round(avgValAcc,6))+"_"+str(round(val_auc,6))+".pth"
    with open('/home/debopriyo/ng_sural/data/DMLFCAD_'+pair_type+'/models/DMLFCAD_'+pair_type+'_Model_Training_Validation_stats_lr_'+str(lr)+'.csv','a') as f:
        f.write(str(epoch+1)+','+str(lr)+','+str(avgTrainLoss)+','+str(avgTrainAcc)+','+str(tr_auc)+','+str(avgValLoss)+','+str(avgValAcc)+','+str(val_auc)+'\n')
    torch.save({
            'state_dict': net.state_dict()
            }, model_path)
    
end = time.time() - start
print('Training completed in {:.0f}m {:.0f}s'.format(end//60,end%60))
sys.stdout.flush()

Epoch: 1 started...
starting tr
fstart
